In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# STEP1: LOADING DATASET

In [ ]:
train_dataset=datasets.MNIST('./data/',
    train=True,
    transform=transforms.ToTensor(),
    target_transform=None,
    download=True)

test_dataset=datasets.MNIST('./data/',
    train=False,
    transform=transforms.ToTensor(),
    target_transform=None,
    download=True)

# STEP2: MAKING DATASET ITERABLE

In [ ]:
batch_size=100
n_iters=3000
num_epochs=n_iters // (len(train_dataset) / batch_size)
num_epochs=int(num_epochs)

train_loader=DataLoader(train_dataset,
    batch_size=batch_size,
    shuffle=True)

test_loader=DataLoader(test_dataset,
    batch_size=batch_size,
    shuffle=False)

# STEP3: CREATE MODEL CLASS

In [ ]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Convolution 1
        self.cnn1=nn.Conv2d(in_channels=1,
            out_channels=16,
            kernel_size=5,
            stride=1,
            padding=0)
        self.relu1=nn.ReLU()
        
        #Max pool1
        self.maxpool1=nn.MaxPool2d(kernel_size=2)
        
        #Convolution 2
        self.cnn2=nn.Conv2d(in_channels=16,
            out_channels=32,
            kernel_size=5,
            stride=1,
            padding=0)
        self.relu2=nn.ReLU()
        
        #Max pool2
        self.maxpool2=nn.MaxPool2d(kernel_size=2)
        
        #Full connected 1
        imagesize=28
        in_features=(1+(imagesize-5)/1)/2
        in_features=(1+(in_features-5)/1)/2
        in_features=int(32*in_features**2)
        out_features=10
        self.fc1=nn.Linear(in_features, out_features)
        
    def forward(self, x):
        # Convolution 1
        out=self.cnn1(x)
        out=self.relu1(out)
        
        #Max pool 1
        out=self.maxpool1(out)
        
        #Convolution 2
        out=self.cnn2(out)
        out=self.relu2(out)
        
        #Max pool 2
        out=self.maxpool2(out)
        
        # Resize
        # out [100, 32, 4, 4]
        out=out.view(out.shape[0], -1)
        out=self.fc1(out)
        return out

# STEP4: INSTANTIATE MODEL CLASS

In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model=CNNModel()
model.to(device)

# STEP5: INSTANTIATE LOSS CLASS

In [ ]:
criterion=nn.CrossEntropyLoss()

# STEP6: INTANTIATE OPTIMIZER CLASS

In [ ]:
learning_rate=0.01
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)

# STEP7: TRAIN THE MODEL

In [ ]:
iter=0
for epoch in range(num_epochs*2):
    for images, labels in train_loader:
        images=images.requires_grad_().to(device)
        labels=labels.to(device)
        optimizer.zero_grad()
        outputs=model(images)
        loss=criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter+=1
        
        if iter%500==0:
            model.eval()
            correct=0
            total=0
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)                
                outputs=model(images)
                _, predicted=torch.max(outputs.data, 1)
                total+=labels.shape[0]
                correct+=(predicted.cpu()==labels.cpu()).sum()
            
            accuracy=100*correct/total
            print('Iteration: {}. Loss: {}. Accuray: {}'.format(iter, loss.item(), accuracy))
            model.train()
        
        